In [1]:
from qiskit_optimization import QuadraticProgram
quadratic_program = QuadraticProgram('schedule')

In [2]:
#лектор,предмет,группа,день,временной слот в каждом дне (lecturer, subject, group, day, time)
L, S, G, D, T = 5,2,2,12,8

#переменная, отвечающая за день, пробегает от 0 до 11 - дни всех двух недель, за исключением двух вс

In [3]:
import numpy as np
quadratic_program = QuadraticProgram('schedule2')
for i in range(L * S * G * D * T):
    quadratic_program.binary_var(name = f'x_{i}')

In [4]:
quadratic_program

<QuadraticProgram: minimize 0, 1920 variables, 0 constraints, 'schedule2'>

In [5]:
#функция для перевода из переменных задачи в индекс битовой строки
def index(lecturer, group, subject, day, time):
    slc = lecturer * 2 * 2 * 12 * 8 + group * 2 * 12 * 8 + subject * 12 * 8 + day * 8 + time
    return slc

In [6]:
from itertools import product

lecturer_range = range(L)
subject_range = range(S)
group_range = range(G)
day_range = range(D)
time_range = range(T)

In [7]:
#В день может быть поставлено не более 6 лекций в расписании группы.

def constraint2(quadratic_program):
    for (day, group) in product(day_range, group_range):
        ind_dict = dict()   
        for (lecturer, subject, time) in product(lecturer_range, subject_range, time_range):
            idx = index(lecturer, group, subject, day, time)
            ind_dict[f'x_{idx}'] = 1
    quadratic_program.linear_constraint(linear = ind_dict, sense="<=", rhs = 6)

constraint2(quadratic_program)

In [8]:
#В день не может быть более двух занятий по одному предмету у группы.

def constraint3(quadratic_program):
    for (group, lecturer, subject, day) in product(group_range, lecturer_range,subject_range,day_range):
        ind_dict = dict()
        for time in time_range:
            idx = index(lecturer, group, subject, day, time)
            ind_dict[f'x_{idx}'] = 1
    quadratic_program.linear_constraint(linear= ind_dict, sense="<=", rhs = 2)   
    
constraint3(quadratic_program)

In [9]:
# Так получилось, что преподаватель n1 не может вести лекции в среду,
# а n2 в понедельник, а n3 никак не может работать по субботам, n4, n5 –
# не могут вести занятия по вторникам. Воскресенье - день без занятий.

def constraint4(quadratic_program):
    
    for (subject, time, group) in product(subject_range, time_range, group_range):
        
        day_lecture_pairs = [([2,8],[0]), ([0,6],[1]), ([5,11],[2]), ([1,7],[3,4])]
        
        for pair in day_lecture_pairs:
            ind_dict = dict()
            for (day, lecturer) in product (pair[0], pair[1]):
                idx = index(lecturer, group, subject, day, time)
                quadratic_program.linear_constraint({f'x_{idx}': 1}, sense="==", rhs = 0)
                                   
constraint4(quadratic_program)

In [10]:
# По требованию учебной программы для каждого предмета должно
# быть выделено 4 часа за период T = 2 недели для каждой группы.

def constraint5(quadratic_program):
    for(group,lecturer,subject) in product(group_range,lecturer_range,subject_range):
        ind_dict = dict()
        for (day, time) in product(day_range, time_range):
            idx = index(lecturer, group, subject, day, time)
            ind_dict[f'x_{idx}'] = 1
        quadratic_program.linear_constraint(linear= ind_dict, sense="==", rhs = 4)

constraint5(quadratic_program)

In [11]:
#Каждый преподаватель имеет ограниченное число часов в неделю на работу E ≤ 20
#(ALWAYS TRUE)

def constraint6(quadratic_program):
    
    ranges = day_range[:6], day_range[6:]
    for rg in ranges:
        for lecturer, day in product(lecturer_range, rg):
            ind_dict = dict()
            for (subject, group, time) in product(subject_range, group_range, time_range):
                idx = index(lecturer, group, subject, day, time)
                ind_dict[f'x_{idx}'] = 1
            quadratic_program.linear_constraint(linear= ind_dict, sense="<=", rhs = 20)
                                  
constraint6(quadratic_program)

In [12]:
def constraint7(quadratic_program):
    ind_dict = dict()
    for(group,lecturer,subject,day,time) in product(group_range,lecturer_range,subject_range,day_range,time_range):
        idx = index(lecturer, group, subject, day, time)
        ind_dict[f'x_{idx}'] = 1
    quadratic_program.linear_constraint(linear= ind_dict, sense="==", rhs = 40)
    
constraint7(quadratic_program)       

In [13]:


#ограничение из здравого смысла: ни в какое время у одной группы не может быть две пары сразу
def dops1(quadratic_program):
    for (group,day,time) in product(group_range,day_range,time_range):
        ind_dict = dict()
        for (lecturer, subject) in product(lecturer_range, subject_range):
            idx = index(lecturer, group, subject, day, time)
            ind_dict[f'x_{idx}'] = 1
        quadratic_program.linear_constraint(linear= ind_dict, sense="<=", rhs = 1)
        
dops1(quadratic_program)

In [14]:
#Ограничение из здравого смысла: Если лектор ведёт какую-то пару, то второй его курс не может вестись в то же время

def dops2(quadratic_program):
    
    for(day,lecturer,time) in product(day_range,lecturer_range,time_range):                        
        idx1 = index(lecturer, 0, 1, day, time)
        idx2 = index(lecturer, 1, 0, day, time)
        quadratic_program.linear_constraint({f'x_{idx1}':1, f'x_{idx2}': 1}, sense="<=", rhs=1)
                
dops2(quadratic_program)

In [15]:
print(quadratic_program.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: schedule2

Minimize
 obj:
Subject To
 c0: x_280 + x_281 + x_282 + x_283 + x_284 + x_285 + x_286 + x_287 + x_376
     + x_377 + x_378 + x_379 + x_380 + x_381 + x_382 + x_383 + x_664 + x_665
     + x_666 + x_667 + x_668 + x_669 + x_670 + x_671 + x_760 + x_761 + x_762
     + x_763 + x_764 + x_765 + x_766 + x_767 + x_1048 + x_1049 + x_1050 + x_1051
     + x_1052 + x_1053 + x_1054 + x_1055 + x_1144 + x_1145 + x_1146 + x_1147
     + x_1148 + x_1149 + x_1150 + x_1151 + x_1432 + x_1433 + x_1434 + x_1435
     + x_1436 + x_1437 + x_1438 + x_1439 + x_1528 + x_1529 + x_1530 + x_1531
     + x_1532 + x_1533 + x_1534 + x_1535 + x_1816 + x_1817 + x_1818 + x_1819
     + x_1820 + x_1821 + x_1822 + x_1823 + x_1912 + x_1913 + x_1914 + x_1915
     + x_1916 + x_1917 + x_1918 + x_1919 <= 6
 c1: x_1912 + x_1913 + x_1914 + x_1915 + x_1916 + x_1917 + x_1918 + x_1919 <= 2
 c2: x_16 = 0
 c3: x_64 = 0
 c4: x_384 = 0
 c5: x_432 = 0
 c6: 

In [16]:
quadratic_program

<QuadraticProgram: minimize 0, 1920 variables, 1075 constraints, 'schedule2'>

In [17]:
from qiskit_optimization.converters import InequalityToEquality
i2eq = InequalityToEquality()
eq_quadratic_problem = i2eq.convert(quadratic_program)


In [18]:
eq_quadratic_problem

<QuadraticProgram: minimize 0, 2654 variables, 1075 constraints, 'schedule2'>

In [ ]:
from qiskit_optimization.converters import LinearEqualityToPenalty
eq2p = LinearEqualityToPenalty(penalty=0.5)
penalty_problem = eq2p.convert(eq_quadratic_problem)

In [ ]:
penalty_problem

In [ ]:
#print(penalty_problem.export_as_lp_string())

In [ ]:
from qiskit_optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
qubo_problem = conv.convert(eq_quadratic_problem)

In [ ]:
#print(qubo_problem.export_as_lp_string())

In [ ]:
#docplex чтобы достать матрицу кубо
from qiskit_optimization.translators import *
dp = to_docplex_mp(qubo_problem)

In [ ]:
#dp.prettyprint()

In [ ]:
N = dp.number_of_binary_variables
N

In [ ]:
QuboMatrix = np.zeros((N,N))

In [ ]:
#Составление матрицы
a = dp.get_objective_expr()

#Проходимся про квадратичным слагаемым, добавляем коэффициенты в матрицу
for var1 in dp.iter_binary_vars():
    for var2 in dp.iter_binary_vars():    
            coeff = a.get_quadratic_coefficient(var1,var2) 
            QuboMatrix[var1.index,var2.index] = coeff
 
#проход по линейным слагаемым, учитывая их как квадратичные
for var, coeff in a.iter_terms():   
    QuboMatrix[var.index,var.index] += coeff   

In [ ]:
QuboMatrix.shape

In [ ]:
from scipy import sparse

In [ ]:
SpQuboMatrix =  sparse.csr_matrix(QuboMatrix)  

In [ ]:
SpQuboMatrix

In [ ]:
#импорт в файл для отправки на аннилер
import pickle
from scipy import sparse

with open('QuboMatrix.pkl', 'wb') as file:
    pickle.dump(SpQuboMatrix,file)

In [ ]:
#test
with open('QuboMatrix.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data